In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.preprocessing import label_binarize

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Load the dataset
data = pd.read_csv("marketing_campaign.csv")

In [5]:
from datetime import date, datetime

In [6]:
data['Year_Birth'] = pd.to_datetime(data['Year_Birth'], format ='%Y')
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])

In [7]:
data['Income'].fillna(data['Income'].median(), inplace = True)

In [8]:
data['Age'] = (data["Dt_Customer"].dt.year.max()) - (data['Year_Birth'].dt.year)
data['Years_Customer'] = (data["Dt_Customer"].dt.year.max()) - (data['Dt_Customer'].dt.year)
data['Days_Customer'] = (data["Dt_Customer"].max()) - (data['Dt_Customer'])

# Total amount spent on products
data['TotalMntSpent'] = data['MntWines'] + data['MntFruits'] + data['MntMeatProducts'] + data['MntFishProducts'] + data['MntSweetProducts'] + data['MntGoldProds']

# Total number of purchases made
data['TotalNumPurchases'] = data['NumWebPurchases'] + data['NumCatalogPurchases'] + data['NumStorePurchases'] + data['NumDealsPurchases']

# Total number of accepted campaigns
data['Total_Acc_Cmp'] = data['AcceptedCmp1'] +data['AcceptedCmp2'] + data['AcceptedCmp3'] + data['AcceptedCmp4'] + data['AcceptedCmp5'] + data['Response']

# adding columns about the day, month and year cutomer joined
data['Year_Joined'] = data['Dt_Customer'].dt.year
data['Month_Joined'] = data['Dt_Customer'].dt.strftime("%B")
data['Day_Joined'] = data['Dt_Customer'].dt.day_name()

# dividing age into groups
data['Age_Group'] = pd.cut(x = data['Age'], bins = [17, 24, 44,64, 150],
                         labels = ['Young adult','Adult','Middel Aged','Senior Citizen'])
# Total children living in the household
data["Children"] = data["Kidhome"] +  data["Teenhome"]

#Deriving living situation by marital status
data["Partner"]=data["Marital_Status"].replace({"Married":"Yes", "Together":"Yes", "Absurd":"No", "Widow":"No", "YOLO":"No", "Divorced":"No", "Single":"No","Alone":"No"})

#Segmenting education levels in three groups
data["Education_Level"]=data["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})


In [9]:
data.drop(['ID','Z_CostContact','Z_Revenue','Year_Birth','Dt_Customer'], axis=1, inplace=True)

In [10]:
df1 = data.copy() # make a copy
df1.drop(['Education','Marital_Status','Years_Customer','Year_Joined','Month_Joined','Day_Joined'], axis=1, inplace=True)

In [11]:
num_col = df1.select_dtypes(include = np.number).columns
for col in num_col:
    q1 = df1[col].quantile(0.25)
    q3 = df1[col].quantile(0.75)
    iqr = q3-q1
    ll = q1-(1.5*iqr)
    ul = q3+(1.5*iqr)
    for ind in df1[col].index:
        if df1.loc[ind,col]>ul:
            df1.loc[ind,col]=ul
        elif df1.loc[ind,col]<ll:
            df1.loc[ind,col]=ll
        else:
            pass
print("Outliers have been taken care of")

Outliers have been taken care of


In [12]:
subset = df1[['Income','Kidhome','Teenhome','Age','Partner','Education_Level']]
print('This is the data we will use for clustering:')
subset.head()

This is the data we will use for clustering:


,Income,Kidhome,Teenhome,Age,Partner,Education_Level
0,58138.0,0,0,57,No,Graduate
1,46344.0,1,1,60,No,Graduate
2,71613.0,0,0,49,Yes,Graduate
3,26646.0,1,0,30,Yes,Graduate
4,58293.0,1,0,33,Yes,Postgraduate


In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
# I am not scaling the kidhome, teenhome cols, cause thire min, max lies between 0 & 2
num_cols = ['Income','Age']
numeric_pipeline = make_pipeline(StandardScaler())
ord_cols = ['Education_Level']
ordinal_pipeline = make_pipeline(OrdinalEncoder(categories=[['Undergraduate','Graduate','Postgraduate']]))
nom_cols = ['Partner']
nominal_pipeline = make_pipeline(OneHotEncoder())
transformer = ColumnTransformer(transformers=[('num',numeric_pipeline,num_cols),
                                           ('ordinal', ordinal_pipeline,ord_cols),
                                              ('nominal' ,nominal_pipeline,nom_cols)
                                             ])
transformer

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 ['Income', 'Age']),
                                ('ordinal',
                                 Pipeline(steps=[('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Undergraduate',
                                                                              'Graduate',
                                                                              'Postgraduate']]))]),
                                 ['Education_Level']),
                                ('nominal',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['Partner'])])

In [14]:
transformed = transformer.fit_transform(subset)
print('Data has been Transformed')

Data has been Transformed


In [15]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=42)

subset['Clusters'] = kmeans.fit_predict(transformed)

In [16]:
import numpy
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier

In [17]:
x = subset.drop('Clusters', axis=1)
y = subset['Clusters']

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [19]:
# adding GradientBoostingClassifier to transformer pipeline
final_pipeline = make_pipeline(transformer, RandomForestClassifier())

In [20]:
final_pipeline.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Income', 'Age']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('ordinalencoder',
                                                                   OrdinalEncoder(categories=[['Undergraduate',
                                                                                               'Graduate',
                                                                                               'Postgraduate']]))]),
                                                  ['Education_Level']),
                                                 ('nominal',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Partner'])])),
                ('randomforestclassifier', RandomForestClassifier())])

In [21]:
final_pipeline.score(x_test, y_test)

0.96875

In [22]:
import pickle
filename = 'classifier.pkl'
pickle.dump(final_pipeline, open(filename,'wb'))

In [23]:
!pip install -q streamlit
!npm install -g localtunnel -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.513s


In [36]:
%%writefile customer_segmentation_app.py

# Import necessary libraries
import pickle
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# Load the model
classifier = pickle.load(open('classifier.pkl', 'rb'))

# Set the page title and icon
st.set_page_config(page_title='Customer Segmentation Web App', page_icon='📊')

# Define custom Bootstrap-style CSS
st.markdown(
    """
    <style>
    /* Add your custom CSS here */
    .stApp {
        background: #f8f9fa;
    }
    .stButton {
        background-color: #007BFF;
        color: #FFFFFF;
        font-weight: bold;
    }
    .stButton:hover {
        background-color: #0056b3;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Set the app title with a colorful background
st.title('📈 Customer Segmentation Web App')
st.markdown('## Analyze and Visualize Customer Segments')

# Customer segmentation function
def segment_customers(input_data):
    prediction = classifier.predict(pd.DataFrame(input_data, columns=['Income', 'Kidhome', 'Teenhome', 'Age', 'Partner', 'Education_Level']))
    return prediction

# Sidebar for user input
st.sidebar.markdown('### Enter Customer Information:')
Income = st.sidebar.number_input("Household Income", value=50000, step=1000)
Kidhome = st.sidebar.selectbox("Number of Kids in Household", (0, 1, 2))
Teenhome = st.sidebar.selectbox("Number of Teens in Household", (0, 1, 2))
Age = st.sidebar.slider("Age", 18, 85, 30)
Partner = st.sidebar.radio("Living With Partner?", ('Yes', 'No'))
Education_Level = st.sidebar.radio("Education Level", ("Undergraduate", "Graduate", "Postgraduate"))

# When 'Segment Customer' is clicked, make the prediction and display the result
if st.sidebar.button("Segment Customer"):
    prediction = segment_customers([[Income, Kidhome, Teenhome, Age, Partner, Education_Level]])
    result = prediction[0]

    st.markdown('### Customer Segment:')
    if result == 0:
        st.success("Cluster 0")
    elif result == 1:
        st.warning("Cluster 1")
    elif result == 2:
        st.info("Cluster 2")
    elif result == 3:
        st.error("Cluster 3")

    # Visualize the customer segments using a doughnut chart
    segment_counts = [0, 0, 0, 0]
    segment_counts[result] = 1

    fig = px.pie(
        values=segment_counts,
        names=["Cluster 0", "Cluster 1", "Cluster 2", "Cluster 3"],
        hole=0.4,
    )
    fig.update_traces(textinfo='percent+label')
    fig.update_layout(title_text="Customer Segment Distribution")

    st.plotly_chart(fig)

# Explanation of the input fields
st.sidebar.markdown('### Input Explanations:')
st.sidebar.markdown("Household Income: The annual household income.")
st.sidebar.markdown("Number of Kids in Household: Select the count of kids in the household.")
st.sidebar.markdown("Number of Teens in Household: Select the count of teenagers in the household.")
st.sidebar.markdown("Age: Age of the customer.")
st.sidebar.markdown("Living With Partner: Whether the customer is living with a partner.")
st.sidebar.markdown("Education Level: The highest level of education of the customer.")

# Additional information about the app
st.sidebar.markdown('### About This App:')
st.sidebar.markdown(
    "This web app uses a machine learning model to classify customers into four segments based on their demographic information. "
    "Explore how different factors can influence customer segmentation."
)

# Disclaimer
st.sidebar.markdown('***Disclaimer: This is a demonstration app for educational purposes.***')

# Adjust the width of the main content
st.markdown("<style>div[role='main']{max-width: 850px;}</style>", unsafe_allow_html=True)



Overwriting customer_segmentation_app.py


In [38]:
!wget -q -O - ipv4.icanhazip.com

34.170.125.192


In [ ]:
!streamlit run customer_segmentation_app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.170.125.192:8501

npx: installed 22 in 2.898s
your url is: https://modern-papers-shout.loca.lt
